# Introduction

How many reads are there in the ATAC libraries? Currently there is only raw data present.

In [1]:
import pandas
import numpy
import requests
import gzip
import IPython

In [2]:
from curation_common import *

In [3]:
server = ENCODED('www.encodeproject.org')
server.load_netrc()

In [4]:
query = server.get_json('/search/?type=Experiment&assay_term_name=ATAC-seq')

In [5]:
def get_replicate_tuple(replicate):
    bio = replicate['biological_replicate_number']
    tech = replicate['technical_replicate_number']
    return (bio, tech)

def find_fastqs(files):
    best_reps = {}
    for file in files:
        if file['file_format'] == 'fastq':
            replicate = file['replicate']
            rep_id = get_replicate_tuple(replicate)
            library = replicate['library']
            library_id = library['accession']

            yield (library_id, file['href'])

In [6]:
for row in query['@graph']:
    experiment_id = row['@id']
    experiment = server.get_json(experiment_id)
    for library_id, href in find_fastqs(experiment['files']):
        print(experiment_id, experiment['lab']['@id'], library_id, href)
    

/experiments/ENCSR064IHX/ /labs/ross-hardison/ ENCLB398NWK /files/ENCFF463PIR/@@download/ENCFF463PIR.fastq.gz
/experiments/ENCSR064IHX/ /labs/ross-hardison/ ENCLB094NVU /files/ENCFF360YBL/@@download/ENCFF360YBL.fastq.gz
/experiments/ENCSR136XSY/ /labs/ross-hardison/ ENCLB430KLB /files/ENCFF194NYS/@@download/ENCFF194NYS.fastq.gz
/experiments/ENCSR136XSY/ /labs/ross-hardison/ ENCLB367TWL /files/ENCFF765DWT/@@download/ENCFF765DWT.fastq.gz
/experiments/ENCSR229QKB/ /labs/ross-hardison/ ENCLB577TBZ /files/ENCFF199CEQ/@@download/ENCFF199CEQ.fastq.gz
/experiments/ENCSR229QKB/ /labs/ross-hardison/ ENCLB667KAT /files/ENCFF343AEN/@@download/ENCFF343AEN.fastq.gz
/experiments/ENCSR793RAV/ /labs/ross-hardison/ ENCLB986ZTD /files/ENCFF535AXY/@@download/ENCFF535AXY.fastq.gz
/experiments/ENCSR793RAV/ /labs/ross-hardison/ ENCLB228WLV /files/ENCFF637MAT/@@download/ENCFF637MAT.fastq.gz
/experiments/ENCSR453AWR/ /labs/ross-hardison/ ENCLB148VRF /files/ENCFF217JOA/@@download/ENCFF217JOA.fastq.gz
/experimen

In [7]:
[ x['href'] for x in experiment['files']]

['/files/ENCFF520LZN/@@download/ENCFF520LZN.fastq.gz',
 '/files/ENCFF232DPD/@@download/ENCFF232DPD.fastq.gz',
 '/files/ENCFF589ZNO/@@download/ENCFF589ZNO.fastq.gz',
 '/files/ENCFF949EKP/@@download/ENCFF949EKP.fastq.gz']

In [8]:
experiment['files'][0]['replicate']['biological_replicate_number']

1

In [13]:
class RequestFastq:
    def __init__(self, href):
        self.href= href
        self.request = requests.get(href, stream=True, allow_redirects=True)
        
    def read(self, chunk):
        return self.request.raw.read(chunk)
    
    def close(self):
        self.request.close()
        
def count_fastq_reads(href):
    req = RequestFastq(href)
    
    stream = gzip.GzipFile(fileobj=req, mode='r')
    for i, line in enumerate(stream):
        pass
    
    return (i+1) / 4

In [14]:
count_fastq_reads('https://www.encodeproject.org/files/ENCFF002EXS/@@download/ENCFF002EXS.fastq.gz')

1185211.0